In [6]:
import os
import cv2
import argparse
import numpy as np
import json
from skimage import color
import openslide
import pandas as pd
import glob
import csv
import shutil
# with open('/Volumes/data/Neptune/AI_tubule_biopsyid.csv') as csvfile:
#     reader = csv.DictReader(csvfile)
#     column = [row for row in reader]
# wsi_folders = '/Volumes/data/Neptune/FSGS_MCD/'

In [7]:
tbm_files = glob.glob(f'/Volumes/data/UMich/spatial_transcriptomic/ROI/TBM/*')

for tbm_file in tbm_files:
    geojson_features = []
    bbox_image = os.path.basename(tbm_file).split('.png')[0]
    # x_roi,y_roi= float(bbox_image.split('_')[1]),float(bbox_image.split('_')[2])
    # print(x_roi,y_roi)
    binary_contour = cv2.imread(tbm_file, cv2.IMREAD_GRAYSCALE)
    binary_mask = cv2.threshold(binary_contour, 127, 255, cv2.THRESH_BINARY)[1]
    contours, hierarchy = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    print(tbm_file,len(contours))
    for i, contour in enumerate(contours):
        if len(contour) > 2:
            # Check if the contour has a parent (has hole(s) or not)
            if hierarchy[0][i][3] == -1:
                print(f'Contour {i} - NO parent')
                contour_json = []
                # Extract and format contour coordinates
                contour_coords = np.squeeze(contour).tolist()
                contour_coords.append(contour_coords[0])
                contour_coords = [[(c[0]), (c[1])] for c in contour_coords]
                contour_coords.append(contour_coords[0])
                contour_json.append(contour_coords)
                
                # Check for holes within the contour
                for j, child_hierarchy in enumerate(hierarchy[0]):
                    if child_hierarchy[3] == i:
                        print(f'Contour {j} - is the hole of Contour {i}')
                        contour_coords = np.squeeze(contours[j]).tolist()
                        contour_coords.append(contour_coords[0])
                        contour_coords = [[(c[0]), (c[1])] for c in contour_coords]
                        contour_coords.append(contour_coords[0])
                        contour_json.append(contour_coords)
                        
                geojson_feature = {
                                        "type": "Feature",
                                        "geometry": {
                                            "type": "Polygon",
                                            "coordinates": contour_json
                                        },
                                        "properties": {
                                            "objectType": "annotation",
                                            "classification": {
                                                "name": "TBM_Thickened",
                                                "color": [29, 9, 7]
                                            }
                                        }
                                    }
                geojson_features.append(geojson_feature)
    with open(f'/Volumes/data/UMich/spatial_transcriptomic/ROI/TBM/{bbox_image}.json', 'w') as f:
        json.dump(geojson_features, f)

/Volumes/data/UMich/spatial_transcriptomic/ROI/TBM/Xenium_V1_0021980_PAS_500602_3614.0_21848.0.png 270
Contour 0 - NO parent
Contour 1 - NO parent
Contour 2 - NO parent
Contour 3 - NO parent
Contour 4 - NO parent
Contour 5 - NO parent
Contour 6 - NO parent
Contour 7 - NO parent
Contour 8 - NO parent
Contour 9 - NO parent
Contour 10 - NO parent
Contour 11 - NO parent
Contour 12 - NO parent
Contour 13 - NO parent
Contour 14 - NO parent
Contour 15 - NO parent
Contour 17 - NO parent
Contour 18 - NO parent
Contour 19 - NO parent
Contour 20 - NO parent
Contour 21 - NO parent
Contour 22 - NO parent
Contour 23 - NO parent
Contour 24 - NO parent
Contour 26 - NO parent
Contour 27 - NO parent
Contour 28 - NO parent
Contour 29 - NO parent
Contour 30 - NO parent
Contour 31 - is the hole of Contour 30
Contour 32 - is the hole of Contour 30
Contour 34 - NO parent
Contour 35 - NO parent
Contour 36 - NO parent
Contour 37 - NO parent
Contour 38 - NO parent
Contour 39 - NO parent
Contour 40 - NO parent
C

In [9]:
len(geojson_features)

1025

In [8]:
folders = glob.glob('dataset_BI*/*')

for folder in folders:
    geojson_features = []
    bioid = os.path.basename(folder)
    print(bioid)
    for i in range(len(column)):
        pid = column[i]
        biopsyID = pid['biopsyid']
        if(biopsyID==bioid):
            filename = pid['filename']
            # print(filename)
            if ('.ndpi' in filename):
                id = filename.split('.ndpi')[0]
            if ('.svs' in filename):
                id = filename.split('.svs')[0]
            if ('.NDPI' in filename):
                id = filename.split('.NDPI')[0]
    print(id,filename)

    tubule_files = glob.glob(f'{folder}/segmetation/*tubule*')
    for tubule_file in tubule_files:
        bbox_image = os.path.basename(tubule_file)
        x_roi,y_roi,w_roi,h_roi = float(bbox_image.split('_')[2]),float(bbox_image.split('_')[4]),float(bbox_image.split('_')[6]),float(bbox_image.split('_')[8])
        print(x_roi,y_roi,w_roi,h_roi)
        binary_contour = cv2.imread(tubule_file, cv2.IMREAD_GRAYSCALE)
        binary_mask = cv2.threshold(binary_contour, 127, 255, cv2.THRESH_BINARY)[1]
        contours, hierarchy = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        print(tubule_file,len(contours))
        for contour in contours:
            if (len(contour)>5):
                contour_coords = np.squeeze(contour).tolist()
                contour_coords.append(contour_coords[0])
                contour_coords = [[4*(c[0])+x_roi, 4*(c[1])+y_roi] for c in contour_coords]
                contour_coords.append(contour_coords[0])
                    # Create GeoJSON feature for image contour
                geojson_feature = {
                    "type": "Feature",
                    "geometry": {
                        "type": "Polygon",
                        "coordinates": [contour_coords]
                    },
                    "properties": {
                              "objectType": "annotation",
                              "classification": {
                                "name": "Tubule",
                                "color": [96, 12, 26]
                              }
                            }
                    }
                geojson_features.append(geojson_feature)
    tbm_files = glob.glob(f'{folder}/segmetation/*tbm*')
    for tbm_file in tbm_files:
        bbox_image = os.path.basename(tbm_file)
        x_roi,y_roi,w_roi,h_roi = float(bbox_image.split('_')[2]),float(bbox_image.split('_')[4]),float(bbox_image.split('_')[6]),float(bbox_image.split('_')[8])
        print(x_roi,y_roi,w_roi,h_roi)
        binary_contour = cv2.imread(tbm_file, cv2.IMREAD_GRAYSCALE)
        binary_mask = cv2.threshold(binary_contour, 127, 255, cv2.THRESH_BINARY)[1]
        contours, hierarchy = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        print(tbm_file,len(contours))
        for contour in contours:
            if (len(contour)>5):
                contour_coords = np.squeeze(contour).tolist()
                contour_coords.append(contour_coords[0])
                contour_coords = [[4*(c[0])+x_roi, 4*(c[1])+y_roi] for c in contour_coords]
                contour_coords.append(contour_coords[0])
                    # Create GeoJSON feature for image contour
                geojson_feature = {
                    "type": "Feature",
                    "geometry": {
                        "type": "Polygon",
                        "coordinates": [contour_coords]
                    },
                    "properties": {
                              "objectType": "annotation",
                              "classification": {
                                "name": "TBM_thickened",
                                "color": [12, 12, 96]
                              }
                            }
                    }
                geojson_features.append(geojson_feature)
            # Create GeoJSON object with all features
            # geojson_obj = {
            #             "type": "FeatureCollection",
            #             "features": geojson_features
            #             }

    with open(f'tubule_json/{id}.json','w') as f:
        json.dump(geojson_features, f)


NameError: name 'glob' is not defined

In [ ]:
def nuclear_shapely(json_file):
    with open(json_file) as f:
            # Load the JSON data from the file
            data = json.load(f)
    cells = data['nuc']
    nuclei_contours = []
    for x in cells:
        # print(x)
        info = cells[x]
        contour = info['contour']
        contour.append(contour[0])
        nuclei_contours.append(Polygon(contour))
    return MultiPolygon(nuclei_contours)
def get_tubule_contour(json_file,flag='Neptune'):
    print(json_file)
    return
def get_tubule_contour_bbox_level(tubuleID,tubule_annotation_raw):
    flag = 0
    for tubulename in tubule_annotation_raw:
        if tubulename in f'{tubuleID}.png':
            flag += 1
            tubule_info = tubule_annotation_raw[tubulename]
            primitive_contours_tubule = tubule_info['tubule'][0]['contours']
            polygons_tubule = [Polygon(contour) for contour in primitive_contours_tubule if len(contour)>5]
            multipolygon_tubule = MultiPolygon(polygons_tubule)
            x_min,y_min,x_max,y_max = multipolygon_tubule.bounds
            x_min = int(x_min - 20)
            y_min = int(y_min - 20) 
            x_offset = -x_min
            y_offset = -y_min
            # Apply the offsets using the `translate` function from shapely.affinity
            translated_polygons = [translate(polygon, xoff=x_offset, yoff=y_offset) for polygon in multipolygon_tubule.geoms]
    if flag == 0:
        raise ValueError(f"DID NOT FIND THE MATCH TUBULE {tubuleID}")
    return MultiPolygon(translated_polygons),x_min,y_min
def draw_tubule_nuclei(folder,tubuleID,multipolygon_tubule,nuclei_contours):
    pas = cv2.imread(f'{folder}/patch/{tubuleID}.png')
    for i, polygon in enumerate(multipolygon_tubule.geoms):
        # Extract and adjust polygon coordinates
        contour_coords = [[(coord[0]), (coord[1])] for coord in polygon.exterior.coords]
        # Draw the adjusted polygon coordinates onto the image
        cv2.drawContours(pas, [np.array(contour_coords, dtype=np.int32)], -1, (138, 223, 178), thickness=cv2.FILLED)
    # Iterate through the geometry collection (MultiPolygon or MultiLineString)
    for i, geom in enumerate(nuclei_contours.geoms):
        if geom.geom_type == 'Polygon':
            # Extract exterior coordinates for polygons
            contour_coords = [[coord[0], coord[1]] for coord in geom.exterior.coords]
        else:
            # Skip unsupported geometry types
            continue

        # Draw the adjusted coordinates onto the image
        cv2.drawContours(
            pas, [np.array(contour_coords, dtype=np.int32)], -1, (255, 255, 178), thickness=cv2.FILLED
        )

    return pas

def keep_only_intersected_nuclei_with_tubule(multipolygon_tubule,nuclei_contours):
    multi_polygon_A = multipolygon_tubule

    multi_polygon_B = nuclei_contours

    # Initialize an empty list to store the filtered polygons
    filtered_polygons_B = []

    # Iterate through each polygon in MultiPolygon B
    for polygon_B in multi_polygon_B.geoms:
        # Check if the polygon intersects with any polygon in MultiPolygon A
        if any(polygon_A.buffer(0).intersects(polygon_B.buffer(0)) for polygon_A in multi_polygon_A.geoms):
            # If so, add the polygon to the filtered list
            filtered_polygons_B.append(polygon_B)

    # Create a new MultiPolygon from the filtered list
    filtered_multi_polygon_B = MultiPolygon(filtered_polygons_B)

    
    return filtered_multi_polygon_B
folders = glob.glob(f'{opt.dataroot}/0_2463_A*')
flag = 'NEPTUNE'
for folder in folders:
    print(folder)
    geojson_features = []
    wsiid = os.path.basename(folder).split('.')[0]
    print(wsiid)
    if flag == 'NEPTUNE':
        tubuleanno_file = glob.glob(f'/data/ffan4/AI_tubule/NEPTUNE/NEPTUNE_FSGS_MCD_CORTEX-PTC-GLOM-TUBULE/{wsiid}*/TUBULE/*')[0]
        with open(tubuleanno_file, 'r') as tubuleanno_file:
            tubule_annotation_raw = json.load(tubuleanno_file)
    nuclei_json_files = glob.glob(f'{folder}/pred/json/*')
    for json_file in tqdm(nuclei_json_files):
        tubuleID = os.path.basename(json_file).split('.json')[0]
        multipolygon_tubule,x_min,y_min = get_tubule_contour_bbox_level(tubuleID,tubule_annotation_raw)
        nuclei_contours = nuclear_shapely(json_file)
        filtered_nuclei = keep_only_intersected_nuclei_with_tubule(multipolygon_tubule,nuclei_contours)
        for nuclei_polygon in filtered_nuclei.geoms:
            if nuclei_polygon.geom_type == 'Polygon':
#                 contour = nuclei_polygon.exterior
                # Print the coordinates of the contour
#                 contour_coords = list(contour.coords)
                contour = list(nuclei_polygon.exterior.coords)
    
                geojson_feature = {
                    "type": "Feature",
                    "geometry": {
                        "type": "Polygon",
                        "coordinates": [contour]
                        },
                    "properties": {
                        "objectType": "annotation",
                                "classification": {
                                    "name": f"nuclei_new",
                                    "color": [12, 12, 96]
                                }
                        }
                    }
                geojson_features.append(geojson_feature)
         # Create GeoJSON object with all features
    geojson_obj = {
    "type": "FeatureCollection",
    "features": geojson_features
    }
    # Save combined GeoJSON object to file
    with open(f'{folder}/nuclei.geojson','w') as f:
        json.dump(geojson_obj, f)